In [2]:
from tf_slim.nets import inception_v1
import tensorflow as tf
import tf_slim as slim

In [3]:
import pandas
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import numpy as np

In [4]:
dt = pandas.read_csv("./data/legend.csv")
val = pandas.read_csv("./data/500_picts_satz.csv")
dt['y'] = dt['emotion'].apply(lambda x: x.lower())

In [5]:
l = os.listdir("./grayscale")
Xs = []
ys = []

In [6]:
for item in l:
    entry = dt[dt['image'] == item]
    if (len(entry) == 1):
        img = mpimg.imread("./grayscale/"+item)
        Xs.append(np.array([img]).tolist())
        ys.append(entry['y'])

In [7]:
x = tf.constant(Xs, dtype=tf.float32)

In [9]:
x2 = tf.reshape(x, [12756, 256, 256, 1])

In [68]:
def build(x, y, num_class):
    df = "channels_last"
    model = tf.keras.models.Sequential()
    model.add(
        tf.keras.layers.Conv2D(
            filters=64, kernel_size = 11, 
            strides=4, data_format=df, 
            input_shape = x.shape[1:])
    )
    model.add(
        tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=2, data_format=df)
    )
    model.add(
        tf.keras.layers.Conv2D(filters=192, kernel_size = 5, data_format=df)
    )
    model.add(
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2, data_format=df)
    )
    model.add(
        tf.keras.layers.Conv2D(filters=384, kernel_size=3, data_format=df)
    )
    model.add(
        tf.keras.layers.Conv2D(filters=384, kernel_size=3, data_format=df)
    )
    model.add(
        tf.keras.layers.Conv2D(filters=256, kernel_size=3, data_format=df)
    )
    model.add(
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2, data_format=df)
    )
    model.add(tf.keras.layers.Flatten(data_format=df))
    model.add(tf.keras.layers.Dense(4096, activation="relu"))
    model.add(tf.keras.layers.Dense(4096, activation="relu"))
    model.add(tf.keras.layers.Dense(num_class, activation="softmax"))
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
    return model

In [69]:
model = build(x2, ys, len(dt['y'].unique()))

In [55]:
y = tf.constant(ys)
labels = dt['y'].unique().tolist()
encodes = {labels[x]:x for x in range(len(labels))}
enc_y = list(map(lambda x: encodes[x.iloc[0]], ys))
lab = tf.keras.utils.to_categorical(enc_y, num_classes=len(dt['y'].unique()))

In [70]:
model.fit(x2, lab, epochs=100, verbose=1)

Epoch 1/100
399/399 [==============================] - 210s 525ms/step - loss: 396386.2188 - accuracy: 0.4570
Epoch 2/100
118/399 [=======>......................] - ETA: 2:25 - loss: 0.9438 - accuracy: 0.5209

KeyboardInterrupt: 

In [8]:
# img = mpimg.imread("./grayscale/Jiang_Zemin_0007.jpg")
# img.shape
# plt.imshow(img)

In [58]:
# for item in notin:
#     entry = val[val['image'] == item]
#     if len(entry) == 0: 
#         print("Absent label: ", item)
#     elif len(entry) == 1:
#         print("In separate dataset: ",item)
#     else: 
#         print("Duplicate entry: ", item)

In [10]:
# outputs = None
# end_points = None

# with slim.arg_scope(alexnet.alexnet_v2_arg_scope()):
#     outputs, end_points = alexnet.alexnet_v2(x2, num_classes = len(dt.y.unique()), spatial_squeeze = False)

# inception_v1.inception_v1_base(x2)